In [1]:
# This uses PynPoint on Altair data as a test

# created 2018 Sept. 23 by E.S.

In [1]:
import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
'''
from PynPoint.Util.Multiprocessing import LineProcessingCapsule, \
                                          PcaMultiprocessingCapsule
'''
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ImageResizing, \
                                       ContrastCurveModule

In [2]:
working_place = "./pynpoint_experimentation_altair/working_place/"
input_place = "./pynpoint_experimentation_altair/input_place/"
output_place = "./pynpoint_experimentation_altair/output_place/"

#url = urllib.URLopener()
#url.retrieve("https://people.phys.ethz.ch/~stolkert/BetaPic_NACO_Mp.hdf5",
#             input_place+"BetaPic_NACO_Mp.hdf5")

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/Core/Pypeline.py:219: UserWarning: Configuration file not found. Creating PynPoint_config.ini with default values.
  warnings.warn("Configuration file not found. Creating PynPoint_config.ini with "


In [3]:
# read in science FITS files

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [4]:
# read in PSF reference FITS files (i.e., unsaturated frames)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place+'ref_psf/',
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

In [7]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]
Running FitsReadingModule... [DONE]


In [8]:
pipeline.get_attribute("ref_psf","PIXSCALE",static=True)

0.0107

In [9]:
pipeline.get_attribute("science","PIXSCALE",static=True)

0.0107

In [5]:
# prepare PSF for PCA-based subtraction

# N.b. Stolker says 'PSFpreparationModule is
# not required since the masking will be done 
# by PSFpreparationModule
# after the planet is injected

'''
# N.b. PSFpreparationModule calls PcaPsfSubtractionModule
prep = PSFpreparationModule(name_in="prep",
                            image_in_tag="read_science",
                            image_out_tag="prep",
                            psf_in_tag="ref_psf",
                            image_mask_out_tag=None,
                            mask_out_tag=None,
                            norm=False,
                            resize=2,
                            cent_size=0.15,
                            edge_size=1.1)

pipeline.add_module(prep)
'''

In [6]:
# do PCA-based PSF subtraction (UNIPROCESSOR)
'''
pca = PcaPsfSubtractionModule(pca_numbers=(20, ),
                              name_in="pca",
                              images_in_tag="prep",
                              reference_in_tag="prep",
                              res_mean_tag="residuals")

pipeline.add_module(pca)
'''

In [5]:
# make a contrast curve (UNIPROCESSOR)

# psf_scaling right?
detec = ContrastCurveModule(name_in="contrast",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape",
                            psf_scaling=10,
                            separation=(0.5, 1.0, 0.1), 
                            angle=(0.0, 360.0, 120.0), 
                            magnitude=(7.5, 1.0))

pipeline.add_module(detec)

In [7]:
# make a contrast curve (MULTIPROCESSOR): don't know how yet...
'''
detec = ContrastCurveModule(name_in="contrast",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape")

pipeline.add_module(detec)
'''

In [ ]:
# line processing multiprocess
'''
LineProcessingCapsule(image_in_port, 
                      image_out_port, 
                      num_processors, 
                      function, 
                      function_args, 
                      length_of_processed_data)
'''

In [6]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Running ContrastCurveModule...
Processing position 1 out of 15.

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:164: UserWarning: The number of frames in psf_in_tag does not match with the number of frames in image_in_tag. Using the mean of psf_in_tag as PSF template.
  warnings.warn('The number of frames in psf_in_tag does not match with the number of '


........

...

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:327: UserWarning: Magnitude decreases but false positive fraction increases. Adjusting magnitude to 1.0625 and step size to 0.015625
  "to %s" % (list_mag[-3], mag_step/2.))


....

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/ProcessingModules/DetectionLimits.py:327: UserWarning: Magnitude decreases but false positive fraction increases. Adjusting magnitude to 1.046875 and step size to 0.0078125
  "to %s" % (list_mag[-3], mag_step/2.))


...

KeyboardInterrupt: 

In [5]:
## TESTING
pixscale = pipeline.get_attribute("science", "PIXSCALE")

In [10]:
import PynPoint.Core.DataIO.InputPort as InputPort
in_port = InputPort("contrast_landscape")

ImportError: No module named InputPort

In [5]:
test = pipeline.get_attribute("science", "PARANG")

KeyError: "Can't open attribute (Can't locate attribute: 'parang')"

In [7]:
test = pipeline.get_attribute("contrast_landscape", "History")

KeyError: "Can't open attribute (Can't locate attribute: 'history')"

In [9]:
test = pipeline.get_attribute("science", "CPU")

KeyError: "Can't open attribute (Can't locate attribute: 'cpu')"

In [6]:
pixscale

0.027

In [7]:
residuals = pipeline.get_data("residuals")
pixscale = pipeline.get_attribute("science", "PIXSCALE")
contrastdata = pipeline.get_attribute("contrast_landscape")

size = pixscale*np.size(residuals, 1)/2.

In [8]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("Altair test")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")